In [ ]:
import os
from terrasnek.api import TFC as TFP
import pprint
import pandas
pp = pprint.PrettyPrinter()

In [ ]:

TFE_TOKEN = os.getenv("TFE_TOKEN", "")
TFE_URL = os.getenv("TFE_ADDR", "https://app.terraform.io")  # ex: https://app.terraform.io

api = TFP(TFE_TOKEN, url=TFE_URL)

orgs = api.orgs.list()['data']
print (f"Found {len(orgs)} Organizations")

total_rum = 0
total_workspaces = 0
result = []
for org in orgs:
    org_workspaces_count = 0
    api.set_org(org['id'])
    print (f".: looking up Organization {org['id']}",)
    all_workspaces = api.workspaces.list_all()['data']
    org_workspaces_count += len (all_workspaces)
    total_workspaces += org_workspaces_count
    print (f"Found {org_workspaces_count} Workspaces")
    
    rum = 0
    for workspace in all_workspaces:
        # to add details per workspace
        #runs = api.runs.list_all(workspace['id'])['data']
        #print(workspace['attributes']['name'], workspace['attributes']['resource-count'])
        rum += int(workspace['attributes']['resource-count'])
    print(f"RUM for org {org['id']}: {rum}")
    result.append(dict(Organization_id = org['id'], Workspaces = all_workspaces, Workspace_count = org_workspaces_count, RUM = rum ))
    total_rum += rum



data = pandas.DataFrame(result)

In [ ]:

data

In [ ]:
selected_row = data.loc[data['Organization_id'] == 'emea-se-playground-2019']
workspaces_list = selected_row['Workspaces'].values[0]
result = []
print("Workspaces for 'emea-se-playground-2019':")
for workspace in workspaces_list:
    name = workspace['attributes']['name']
    resource_count = workspace['attributes']['resource-count']
    result.append(dict(Name = name, Resource_count = resource_count))
rum_by_workspace = pandas.DataFrame(result)
rum_by_workspace.sort_values(by=['Resource_count'],ascending=False)


In [ ]:
rum_by_workspace.sum()